# NGS Data formats and QC

## Introduction
There are several file formats for storing Next Generation Sequencing (NGS) data. In this tutorial we will look at some of the most common formats for storing NGS reads and variant data. We will cover the following formats:

__FASTQ__ - This format stores unaligned read sequences with base qualities  
__SAM/BAM__ - This format stores unaligned or aligned reads (text and binary formats)  
__CRAM__ - This format is similar to BAM but has better compression than BAM  
__VCF/BCF__ - Flexible variant call format for storing SNPs, indels, structural variations (text and binary formats)   

Further to understanding the different file formats, it is important to remember that all sequencing platforms have technical limitations that can introduce biases in your sequencing data. Because of this it is very important to check the quality of the data before starting any analysis, whether you are planning to use something you have sequenced yourself or publicly available data. In the second part of this tutorial we will describe how to perform a QC assessment for your NGS data.


In [2]:
mkdir ngs
cd ngs

Now you can follow the instructions in the tutorial from here.

## Let’s get started!
Let's download `samtools`,`bcftools`and `htslib`.

Do this from commandline.

```
sudo yum install samtools bcftools htslib gnuplot
```

*Correction Modules loaded from Cheaha

In [1]:
module load BCFtools/1.12-GCC-10.2.0

In [2]:
module load SAMtools/1.12-GCC-10.2.0

This should return the help message for samtools and bcftools respectively.

## Download the required datasets

In [1]:
wget https://github.com/cb2edu/CB2-101-BioComp/raw/2020/07-NGS_Intro_QC/data/example.tgz
tar -xvzf example.tgz

--2021-10-26 10:07:00--  https://github.com/cb2edu/CB2-101-BioComp/raw/2020/07-NGS_Intro_QC/data/example.tgz
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/cb2edu/CB2-101-BioComp/2020/07-NGS_Intro_QC/data/example.tgz [following]
--2021-10-26 10:07:01--  https://raw.githubusercontent.com/cb2edu/CB2-101-BioComp/2020/07-NGS_Intro_QC/data/example.tgz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28092060 (27M) [application/octet-stream]
Saving to: ‘example.tgz’

100%[======================================>] 28,092,060   102MB/s   in 0.3s   

2021-10-26 10:07:01 (102 MB/s) - ‘example.tgz’ saved [2

## FASTQ
FASTQ is a data format for raw unaligned sequencing reads. It is an extension to the FASTA file format, and includes a quality score for each base. Have a look at the example below, containing two reads:
  
```
@ERR007731.739 IL16_2979:6:1:9:1684/1
CTTGACGACTTGAAAAATGACGAAATCACTAAAAAACGTGAAAAATGAGAAATG
+
BBCBCBBBBBBBABBABBBBBBBABBBBBBBBBBBBBBABAAAABBBBB=@>B
@ERR007731.740 IL16_2979:6:1:9:1419/1
AAAAAAAAAGATGTCATCAGCACATCAGAAAAGAAGGCAACTTTAAAACTTTTC
+
BBABBBABABAABABABBABBBAAA>@B@BBAA@4AAA>.>BAA@779:AAA@A
```
   
We can see that for each read we get four lines:

   1. The read metadata, such as the read ID. Starts with `@` and, for paired-end Illumina reads, is terminated with /1 or /2 to show that the read is the member of a pair. 
   2. The read
   3. Starts with `+` and optionally contains the ID again
   4. The per base [Phred quality score](https://en.wikipedia.org/wiki/Phred_quality_score)

The quality scores range (in theory) from 1 to 94 and are encoded as [ASCII characters](https://en.wikipedia.org/wiki/ASCII). The first 32 ASCII codes are reserved for control characters which are not printable, and the 33rd is reserved for space. Neither of these can be used in the quality string, so we need to subtract 33 from whatever the value of the quality character is. For example, the ASCII code of “A” is 65, so the corresponding quality is:
   
```   
Q = 65 - 33 = 32
```
   
The Phred quality score `Q` relates to the base-calling error probability `P` as
   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; P = 10<sup>-Q/10</sup>   
   
The Phred quality score is a measure of the quality of base calls. For example, a base assigned with a Phred quality score of 30 tells us that there is a 1 in 1000 chance that this base was called incorrectly.

|Phred Quality Score|Probability of incorrect base call|Base call accuracy|
|---|---|---|
|10|1 in 10|90%|
|20|1 in 100|99%|
|30|1 in 1000|99.9%|
|40|1 in 10,000|99.99%|
|50|1 in 100,000|99.999%|
|60|1 in 1,000,000|99.9999%|

The following simple perl command will print the quality score value for an ASCII character. Try changing the "A" to another character, for example one from the quality strings above (e.g. @, = or B).

In [2]:
perl -e 'printf "%d\n",ord("@")-33;'

31


### Exercises
__Q2.1: How many reads are there in the file example.fastq? (Hint: remember that `@` is a possible quality score. Is there something else in the header that is unique?)__

In [3]:
grep -c -i -o @ example.fastq

16


## SAM
[SAM (Sequence Alignment/Map)](https://samtools.github.io/hts-specs/SAMv1.pdf) format was developed by the 1000 Genomes Project group in 2009 and is a unified format for storing read alignments to a reference genome. SAM format is a standard format for storing NGS sequencing reads, base qualities, associated meta-data and alignments of the data to a reference genome. If no reference genome is available, the data can also be stored unaligned.

The files consist of a header section (optional) and an alignment section. The alignment section contains one record (a single DNA fragment alignment) per line describing the alignment between fragment and reference. Each record has 11 fixed columns and optional key:type:value tuples. Open the SAM/BAM file specification document [https://samtools.github.io/hts-specs/SAMv1.pdf](https://samtools.github.io/hts-specs/SAMv1.pdf) either in a web browser or you can find a copy in the QC directory as you may need to refer to it throughout this tutorial. 

Now let us have a closer look at the different parts of the SAM/BAM format. 

### Header Section

The header section of a SAM file looks like:

<insert header text>

```
@HD	VN:1.0	SO:coordinate
@SQ	SN:test_ref	LN:17637
@RG ID:ERR003612 PL:ILLUMINA LB:g1k-sc-NA20538-TOS-1 PI:2000 DS:SRP000540 SM:NA20538 CN:SC
```

Each line in the SAM header begins with an `@`, followed by a two-letter header record type code as defined in the [SAM/BAM format specification document](https://samtools.github.io/hts-specs/SAMv1.pdf). Each record type can contain meta-data captured as a series of key-value pairs in the format of ‘TAG:VALUE’.

#### Read groups
One useful record type is RG which can be used to describe each lane of sequencing. The RG code can be used to capture extra meta-data for the sequencing lane. Some common RG TAGs are:

* ID: SRR/ERR number
* PL: Sequencing platform
* PU: Run name
* LB: Library name
* PI: Insert fragment size
* SM: Individual/Sample
* CN: Sequencing centre

While most of these are self explanitory, insert fragment size may occasionally be negative. This simply indicates that the reads found are overlapping while its size is less than 2 x read length.

### Exercises

Look at the following line from the header of a SAM file and answering the questions that follow: 

```
@RG ID:ERR003612 PL:ILLUMINA LB:g1k-sc-NA20538-TOS-1 PI:2000 DS:SRP000540 SM:NA20538 CN:SC
```
   
__Q3: What does RG stand for?__

RG stands for read group 

__Q4: What is the sequencing platform?__

the sequencing platform is illumina 

__Q5: What is the sequencing centre?__

The sequencing center is SC

__Q6: What is the lane identifier?__

the lane identifier is the RG header entries 

__Q7: What is the expected fragment insert size?__  

The expected fragment insert size is 2000

### Alignment Section
The alignment section of SAM files contains one line per read alignment, an example is

ERR005816.1408831	163	Chr1	19999970	23	40M5D30M2I28M	=	20000147	213 GGTGGGTGGATCACCTGAGATCGGGAGTTTGAGACTAGGTGG...   
<=@A@??@=@A@A>@BAA@ABA:>@<>=BBB9@@2B3<=@A@...   

Each of the lines are composed of multiple columns listed below. The first 11 columns are mandatory.

1. QNAME: Query NAME of the read or the read pair i.e. DNA sequence
2. FLAG: Bitwise FLAG (pairing, strand, mate strand, etc.)
3. RNAME: Reference sequence NAME
4. POS: 1-Based leftmost POSition of clipped alignment
5. MAPQ: MAPping Quality (Phred-scaled)
6. CIGAR: Extended CIGAR string (operations: MIDNSHPX=)
7. MRNM: Mate Reference NaMe (’=’ if same as RNAME)
8. MPOS: 1-Based leftmost Mate POSition
9. ISIZE: Inferred Insert SIZE
10. SEQ: Query SEQuence on the same strand as the reference
11. QUAL: Query QUALity (ASCII-33=Phred base quality)
12. OTHER: Optional fields


### Exercises
Let's have a look at example.sam. This file contains only a subset of the alignment section of a BAM-file that we will look closer at soon. Notice that we can use the standard UNIX operations like __cat__ on this file.

In [3]:
pwd

/data/user/home/mmeghan/Desktop/info302_assignment/Assignment7_NGS_Intro_QC


In [3]:
cat example.sam

ERR003814.1408899	163	1	19999970	23	37M	=	20000147	213	GGTGGGTGGATCACCTGAGATCGGGAGTTTGAGACTA	<=@A@??@=@A@A>@BAA@ABA:>@<>=BBB9@@2B3	X0:i:1	X1:i:20	MD:Z:37	RG:Z:ERR003814	AM:i:0	NM:i:0	SM:i:23	MQ:i:29	XT:A:U	BQ:Z:@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
ERR003762.5016205	163	1	20000006	48	37M	=	20000026	55	AACATGGAGAAACCCCATCTCTACTAAGGTACAAAAA	<A:@AA?7@@6B69<<AB<B7;++?:=;+7;7=@@=%	X0:i:1	X1:i:43	MD:Z:36T0	RG:Z:ERR003762	AM:i:23	NM:i:1	SM:i:23	MQ:i:48	XT:A:U	BQ:Z:@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@DFD@
ERR015472.4248280	83	1	20000009	60	108M	=	19999730	-386	ATGGAGAAACCCCATCTCTACTAAGGTACAAAATTAGATGGGCATGGTGGTGCACACCTGTAGTCCCAGCTACTTGGGAGGCTGAGGCAGGAGAATCGCTTGAACCCG	:<35@;?@-8;1@=A@?@?@@>@A><@A@?@BA@?B<@B>?@AAB??A5>B@?@AACAB?BC?BCBBBBBC@BCCBBBCBBBBBCBBBCBACABAC:AACCAB?A@6?	X0:i:1	X1:i:0	MD:Z:108	RG:Z:ERR015472	AM:i:37	NM:i:0	SM:i:37	MQ:i:60	XT:A:U	BQ:Z:@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@A
ERR015472.13730806	83	1	20000013	60	

__Q8: What is the mapping quality of ERR003762.5016205? (Hint: can you use grep and awk to find this?)__

In [7]:
grep -i "ERR003762.5016205" example.sam | awk '{print $1, $5}'

ERR003762.5016205 48


__Q9: What is the CIGAR string for ERR003814.6979522? (Hint: we will go through the meaning of CIGAR strings in the next section)__

In [8]:
grep -i "ERR003814.6979522" example.sam | awk '{print $1, $6}'

ERR003814.6979522 37M


__Q10: What is the inferred insert size of ERR003814.1408899?__

In [9]:
grep -i "ERR003814.1408899" example.sam | awk '{print $1, $9}'

ERR003814.1408899 213


### CIGAR string
Column 6 of the alignment is the CIGAR string for that alignment. The CIGAR string provides a compact representation of sequence alignment. Have a look at the table below. It contains the meaning of all different symbols of a CIGAR string:

|Symbol |Meaning                                             |
|---    |---                                                 |
|M      |alignment match or mismatch                         |
|=      |sequence match                                      |
|X      |sequence mismatch                                   |
|I      |insertion into the read (sample sequenced)          |
|D      |deletion from the read (sample sequenced)           |
|S      |soft clipping (clipped sequences present in SEQ)    |
|H      |hard clipping (clipped sequences NOT present in SEQ)|
|N      |skipped region from the reference                   |
|P      |padding (silent deletion from padded reference)     |

Below are two examples describing the CIGAR string in more detail.
  
__Example 1:__  
Ref:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ACGTACGTACGTACGT  
Read:&nbsp;&nbsp;ACGT-&nbsp;-&nbsp;-&nbsp;-&nbsp;ACGTACGA  
Cigar: 4M 4D 8M  

The first four bases in the read are the same as in the reference, so we can represent these as 4M in the CIGAR string. Next comes 4 deletions, represented by 4D, followed by 7 alignment matches and one alignment mismatch, represented by 8M. Note that the mismatch at position 16 is included in 8M. This is because it still aligns to the reference.

__Example 2:__  
Ref:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ACTCAGTG-&nbsp;-&nbsp;GT  
Read:&nbsp;&nbsp;ACGCA-&nbsp;TGCAGTtagacgt  
Cigar: 5M 1D 2M 2I 2M 7S  

Here we start off with 5 alignment matches and mismatches, followed by one deletion. Then we have two more alignment matches, two insertions and two more matches. At the end, we have seven soft clippings, 7S. These are clipped sequences that are present in the SEQ (Query SEQuence on the same strand as the reference).

### Exercises
__Q11: What does the CIGAR from Q9 mean?__

The CIGAR from question 9 was 37M meaning that there are 37 allignment match or mismatch  

__Q12: How would you represent the following alignment with a CIGAR string?__  
 
Ref:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ACGT-&nbsp;-&nbsp;-&nbsp;-&nbsp;ACGTACGT  
Read:&nbsp;&nbsp;ACGTACGTACGTACGT  

4M 3I 1M 1I 7M

### Flags
Column 2 of the alignment contains a combination of bitwise FLAGs describing the alignment. The following table contains the information you can get from the bitwise FLAGs:
   
|Hex  |Dec |Flag         |Description                                           |
|---  |--- |---          |---                                                   |
|0x1  |1   |PAIRED       |paired-end (or multiple-segment) sequencing technology|
|0x2  |2   |PROPER_PAIR  |each segment properly aligned according to the aligner|
|0x4  |4   |UNMAP        |segment unmapped                                      |
|0x8  |8   |MUNMAP       |next segment in the template unmapped                 |
|0x10 |16  |REVERSE      |SEQ is reverse complemented                           |
|0x20 |32  |MREVERSE     |SEQ of the next segment in the template is reversed   |
|0x40 |64  |READ1        |the first segment in the template                     |
|0x80 |128 |READ2        |the last segment in the template                      |
|0x100|256 |SECONDARY    |secondary alignment                                   |
|0x200|512 |QCFAIL       |not passing quality controls                          |
|0x400|1024|DUP          |PCR or optical duplicate                              |
|0x800|2048|SUPPLEMENTARY|supplementary alignment                               |

For example, if you have an alignment with FLAG set to 113, this can only be represented by decimal codes `64 + 32 + 16 + 1`, so we know that these four flags apply to the alignment and the alignment is paired-end, reverse complemented, sequence of the next template/mate of the read is reversed and the read aligned is the first segment in the template.

#### Primary, secondary and supplementary alignments
A read that aligns to a single reference sequence (including insertions, deletions, skips and clipping but not direction changes), is a __linear alignment__. If a read cannot be represented as a linear alignment, but instead is represented as a group of linear alignments without large overlaps, it is called a __chimeric alignment__. These can for instance be caused by structural variations. Usually, one of the linear alignments in a chimeric alignment is considered to be the __representative__ alignment, and the others are called __supplementary__.

Sometimes a read maps equally well to more than one spot. In these cases, one of the possible alignments is marked as the __primary__ alignment and the rest are marked as __secondary__ alignments.

## BAM
BAM (Binary Alignment/Map) format, is a compressed binary version of SAM. This means that, while SAM is human readable, BAM is only readable for computers. BAM was developed for fast processing and random access. To achieve this, BGZF (Block GZIP) compression is used for indexing. BAM files can be viewed using samtools, and will then have the same format as a SAM file.  The key features of BAM are:

* Can store alignments from most mappers
* Supports multiple sequencing technologies
* Supports indexing for quick retrieval/viewing
* Compact size (e.g. 112Gbp Illumina = 116GB disk space)
* Reads can be grouped into logical groups e.g. lanes, libraries, samples
* Widely supported by variant calling packages and viewers

Since BAM is a binary format, we can't use the standard UNIX operations directly on this file format. __Samtools__ is a set of programs for interacting with SAM and BAM files. Using the samtools view command, print the header of the BAM file:

In [21]:
samtools view -H NA20538.bam

@HD	VN:1.0	GO:none	SO:coordinate
@SQ	SN:1	LN:249250621	AS:NCBI37	UR:ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/human_g1k_v37.fasta.gz	M5:1b22b98cdeb4a9304cb5d48026a85128	SP:Human
@SQ	SN:2	LN:243199373	AS:NCBI37	UR:ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/human_g1k_v37.fasta.gz	M5:a0d9851da00400dec1098a9255ac712e	SP:Human
@SQ	SN:3	LN:198022430	AS:NCBI37	UR:ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/human_g1k_v37.fasta.gz	M5:fdfd811849cc2fadebc929bb925902e5	SP:Human
@SQ	SN:4	LN:191154276	AS:NCBI37	UR:ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/human_g1k_v37.fasta.gz	M5:23dccd106897542ad87d2765d28a19a1	SP:Human
@SQ	SN:5	LN:180915260	AS:NCBI37	UR:ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/human_g1k_v37.fasta.gz	M5:0740173db9ffd264d728f32784845cd7	SP:Human
@SQ	SN:6	LN:171115067	AS:NCBI37	UR:ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/human_g1k_v37.fasta.gz	M5:1d3a93a248d92a729ee764823

@SQ	SN:GL000243.1	LN:43341	AS:NCBI37	UR:ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/human_g1k_v37.fasta.gz	M5:cc34279a7e353136741c9fce79bc4396	SP:Human
@SQ	SN:GL000242.1	LN:43523	AS:NCBI37	UR:ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/human_g1k_v37.fasta.gz	M5:2f8694fc47576bc81b5fe9e7de0ba49e	SP:Human
@SQ	SN:GL000230.1	LN:43691	AS:NCBI37	UR:ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/human_g1k_v37.fasta.gz	M5:b4eb71ee878d3706246b7c1dbef69299	SP:Human
@SQ	SN:GL000237.1	LN:45867	AS:NCBI37	UR:ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/human_g1k_v37.fasta.gz	M5:e0c82e7751df73f4f6d0ed30cdc853c0	SP:Human
@SQ	SN:GL000233.1	LN:45941	AS:NCBI37	UR:ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/human_g1k_v37.fasta.gz	M5:7fed60298a8d62ff808b74b6ce820001	SP:Human
@SQ	SN:GL000204.1	LN:81310	AS:NCBI37	UR:ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/human_g1k_v37.fasta.gz	M5:efc49c871536fa8d79cb0a06fa73

@PG	ID:bwa	VN:0.5.5
@PG	ID:samtools	PN:samtools	PP:bwa	VN:1.12	CL:samtools view -H NA20538.bam


### Exercises
__Q13: What version of the human assembly was used to perform the alignments? (Hint: Can you spot this somewhere in the @SQ records?)__

AS:NCBI37 

__Q14: How many lanes are in this BAM file? (Hint: Do you recall what RG represents?)__

16

__Q15: What version of bwa was used to align the reads? (Hint: is there anything in the @PG record that looks like it could be a version tag?)__  

VN:0.5.5

The output from running samtools view on a BAM file without any options is a headerless SAM file. This gets printed to STDOUT in the terminal, so we will want to pipe it to something. Let's have a look at the first read of the BAM file:

In [20]:
samtools view NA20538.bam | head -n 1

ERR003814.1408899	163	1	19999970	23	37M	=	20000147	213	GGTGGGTGGATCACCTGAGATCGGGAGTTTGAGACTA	<=@A@??@=@A@A>@BAA@ABA:>@<>=BBB9@@2B3	X0:i:1	X1:i:20	MD:Z:37	RG:Z:ERR003814	AM:i:0	NM:i:0	SM:i:23	MQ:i:29	XT:A:U	BQ:Z:@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
samtools view: writing to standard output failed: Broken pipe
samtools view: error closing standard output: -1


__Q16: What is the name of the first read? (Hint: have a look at the alignment section if you can't recall the different fields)__

ERR003814.1408899

__Q17: What position does the alignment of the read start at?__

19999970

## Indexing
To allow for fast random access of regions in BAM and CRAM files, they can be indexed. The files must first be coordinate-sorted rather that sorted by read name. This can be done using __samtools sort__. If no options are supplied, it will by default sort by the left-most position of the reference.

In [22]:
samtools sort -o NA20538_sorted.bam NA20538.bam

Now we can use __samtools index__ to create an index file (.bai) for our sorted BAM file:

In [23]:
samtools index NA20538_sorted.bam

In [24]:
ls

1kg.bcf			   FileTypeNotes.ipynb	   ngs_overview.pdf
data_formats_and_qc.ipynb  NA20538.bam		   ngs_qc.pdf
example.fastq		   NA20538_sorted.bam	   ngs_qc.Rmd
example.sam		   NA20538_sorted.bam.bai  SAMv1.pdf
example.tgz		   ngs


To look for reads mapped to a specific region, we can use __samtools view__ and specify the region we are interested in as: RNAME[:STARTPOS[-ENDPOS]]. For example, if we wanted to look at all the reads mapped to a region called chr4, we could use:

``samtools view alignment.bam chr4``

To look at the region on chr4 beginning at position 1,000,000 and ending at the end of the chromosome, we can do:

  ``samtools view alignment.bam chr4:1000000``
  
And to explore the 1001bp long region on chr4 beginning at position 1,000 and ending at position 2,000, we can use:

  ``samtools view alignment.bam chr4:1000-2000``

### Exercises 
__Q18: How many reads are mapped to region 20025000-20030000 on chromosome 1?__

In [25]:
samtools view NA20538_sorted.bam 1:20025000-20030000 | wc -l

320


## VCF
The VCF file format was introduced to store variation data. VCF consists of tab-delimited text and is parsable by standard UNIX commands which makes it flexible and user-extensible. 

### VCF header
The VCF header consists of meta-information lines (starting with `##`) and a header line (starting with `#`). All meta-information lines are optional and can be put in any order, except for _fileformat_. This holds the information about which version of VCF is used and must come first. 

The meta-information lines consist of key=value pairs. Examples of meta-information lines that can be included are ##INFO, ##FORMAT and ##reference. The values can consist of multiple fields enclosed by `<>`. More information about these fields is available in the VCF specification [http://samtools.github.io/hts-specs/VCFv4.3.pdf](http://samtools.github.io/hts-specs/VCFv4.3.pdf). This can be accessed using a web browser and there is a copy in the QC directory.

#### Header line
The header line starts with `#` and consists of 8 required fields:

1. CHROM: an identifier from the reference genome
2. POS: the reference position
3. ID: a list of unique identifiers (where available)
4. REF: the reference base(s)
5. ALT: the alternate base(s)
6. QUAL: a phred-scaled quality score
7. FILTER: filter status
8. INFO: additional information

If the file contains genotype data, the required fields are also followed by a FORMAT column header, and then a number of sample IDs. The FORMAT field specifies the data types and order. Some examples of these data types are:

* GT: Genotype, encoded as allele values separated by either / or |
* DP: Read depth at this position for this sample
* GQ: Conditional genotype quality, encoded as a phred quality

### Body
In the body of the VCF, each row contains information about a position in the genome along with genotype information on samples for each position, all according to the fields in the header line.

## BCF

BCF is a compressed binary representation of VCF.

VCF can be compressed with BGZF (bgzip) and  indexed with TBI or CSI (tabix), but even compressed it can still be very big. For example, a compressed VCF with 3781 samples of human data will be 54 GB for chromosome 1, and 680 GB for the whole genome. VCFs can also be slow to parse, as text conversion is slow. The main bottleneck is the "FORMAT" fields. For this reason the BCF format was developed. 

In BCF files the fields are rearranged for fast access. The following images show the process of converting a VCF file into a BCF file. 

Bcftools comprises a set of programs for interacting with VCF and BCF files. It can be used to convert between VCF and BCF and to view or extract records from a region.

### bcftools view  
Let's have a look at the header of the file 1kg.bcf in the data directory. Note that bcftools uses __`-h`__ to print only the header, while samtools uses __`-H`__ for this. 

In [26]:
bcftools view -h 1kg.bcf

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##samtoolsVersion=1.2-29-gca44680+htslib-1.2.1-50-g66e5ef4
##samtoolsCommand=samtools mpileup -gu -t DP -I -f ref/human/GRCh37/hs37d5.fa -b bams.chr20.list
##reference=file:///ref/human/GRCh37/hs37d5.fa
##bcftools_callVersion=1.2-208-gcdcf7e1+htslib-1.2.1-158-g6876a07
##bcftools_callCommand=call -mv -Oz -o chr20.dp.vcf.gz
##contig=<ID=1,length=249250621>
##contig=<ID=2,length=243199373>
##contig=<ID=3,length=198022430>
##contig=<ID=4,length=191154276>
##contig=<ID=5,length=180915260>
##contig=<ID=6,length=171115067>
##contig=<ID=7,length=159138663>
##contig=<ID=8,length=146364022>
##contig=<ID=9,length=141213431>
##contig=<ID=10,length=135534747>
##contig=<ID=11,length=135006516>
##contig=<ID=12,length=133851895>
##contig=<ID=13,length=115169878>
##contig=<ID=14,length=107349540>
##contig=<ID=15,length=102531392>
##contig=<ID=16,length=90354753>
##contig=<ID=17,length=81195210>
##contig=<ID=18,length=78077248>
##c

Similarly to BAM, BCF supports random access, that is, fast retrieval from a given region. For this, the file must be indexed:

In [27]:
bcftools index 1kg.bcf

Now we can extract all records from the region 20:24042765-24043073, using the __`-r`__ option. The __`-H`__ option will make sure we don't include the header in the output:

In [28]:
bcftools view -H -r 20:24042765-24043073 1kg.bcf

20	24042765	.	A	T	445	.	AN=100;AC=5;AC_Het=5;AC_Hom=0;AC_Hemi=0	GT:PL:DP	0/1:42,3,0:1	0/0:0,18,166:6	0/0:0,27,255:9	0/1:158,0,93:10	0/1:121,0,87:7	0/0:0,15,166:5	0/0:0,6,100:2	0/0:0,39,255:13	0/0:0,15,155:5	0/0:0,21,215:7	0/0:0,12,138:4	0/0:0,24,212:8	0/0:0,12,127:4	0/0:0,24,209:8	0/0:0,15,185:5	0/0:0,27,255:9	0/0:0,9,110:3	0/0:0,9,95:3	0/0:0,18,165:6	0/0:0,24,216:8	0/0:0,27,247:9	0/0:0,3,30:1	0/0:0,9,77:3	0/0:0,12,119:4	0/0:0,21,201:7	0/0:0,12,127:4	0/0:0,45,255:15	0/0:0,3,32:1	0/0:0,30,255:10	0/0:0,21,216:7	0/0:0,15,162:5	0/1:53,0,130:6	0/0:0,12,142:4	0/0:0,12,114:4	0/0:0,33,255:11	0/0:0,39,255:13	0/0:0,15,174:5	0/0:0,12,120:4	0/0:0,24,228:8	0/0:0,30,255:10	0/1:166,0,42:8	0/0:0,27,248:9	0/0:0,6,78:2	0/0:0,45,255:15	0/0:0,9,99:3	0/0:0,15,143:5	0/0:0,33,255:11	0/0:0,15,158:5	0/0:0,9,111:3	0/0:0,12,130:4
20	24042793	.	G	A	734	.	AN=100;AC=7;AC_Het=7;AC_Hom=0;AC_Hemi=0	GT:PL:DP	0/1:35,3,0:1	0/0:0,15,151:5	0/0:0,36,255:12	0/1:176,0,138:12	0/1:120,0,87:7	0/0:0,9,106:3	0/0:0,12,166:4	0/0:0,3

### bcftools query  
The versatile __bcftools query__ command can be used to extract any VCF field. Combined with standard UNIX commands, this gives a powerful tool for quick querying of VCFs. Have a look at the usage options:

In [29]:
bcftools query -h


About:   Extracts fields from VCF/BCF file and prints them in user-defined format
Usage:   bcftools query [options] <A.vcf.gz> [<B.vcf.gz> [...]]

Options:
    -e, --exclude <expr>              exclude sites for which the expression is true (see man page for details)
    -f, --format <string>             see man page for details
    -H, --print-header                print header
    -i, --include <expr>              select sites for which the expression is true (see man page for details)
    -l, --list-samples                print the list of samples and exit
    -o, --output <file>               output file name [stdout]
    -r, --regions <region>            restrict to comma-separated list of regions
    -R, --regions-file <file>         restrict to regions listed in a file
    -s, --samples <list>              list of samples to include
    -S, --samples-file <file>         file of samples to include
    -t, --targets <region>            similar to -r but streams rather than index-

: 1

Let's try out some useful options. As you can see from the usage, __`-l`__ will print a list of all the samples in the file. Give this a go:

In [30]:
bcftools query -l 1kg.bcf

HG00096
HG00097
HG00099
HG00100
HG00101
HG00102
HG00103
HG00105
HG00106
HG00107
HG00108
HG00109
HG00110
HG00111
HG00112
HG00113
HG00114
HG00115
HG00116
HG00117
HG00118
HG00119
HG00120
HG00121
HG00122
HG00123
HG00124
HG00125
HG00126
HG00127
HG00128
HG00129
HG00130
HG00131
HG00132
HG00133
HG00136
HG00137
HG00138
HG00139
HG00140
HG00141
HG00142
HG00143
HG00145
HG00146
HG00148
HG00149
HG00150
HG00151


Another very useful option is __`-s`__ which allows you to extract all the data relating to a particular sample. This is a [common option](http://samtools.github.io/bcftools/bcftools.html#common_options) meaning it can be used for many bcftools commands, like `bcftools view`. Try this for sample HG00131:

In [4]:
bcftools view -s HG00131 1kg.bcf | head -n 50

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##samtoolsVersion=1.2-29-gca44680+htslib-1.2.1-50-g66e5ef4
##samtoolsCommand=samtools mpileup -gu -t DP -I -f ref/human/GRCh37/hs37d5.fa -b bams.chr20.list
##reference=file:///ref/human/GRCh37/hs37d5.fa
##bcftools_callVersion=1.2-208-gcdcf7e1+htslib-1.2.1-158-g6876a07
##bcftools_callCommand=call -mv -Oz -o chr20.dp.vcf.gz
##contig=<ID=1,length=249250621>
##contig=<ID=2,length=243199373>
##contig=<ID=3,length=198022430>
##contig=<ID=4,length=191154276>
##contig=<ID=5,length=180915260>
##contig=<ID=6,length=171115067>
##contig=<ID=7,length=159138663>
##contig=<ID=8,length=146364022>
##contig=<ID=9,length=141213431>
##contig=<ID=10,length=135534747>
##contig=<ID=11,length=135006516>
##contig=<ID=12,length=133851895>
##contig=<ID=13,length=115169878>
##contig=<ID=14,length=107349540>
##contig=<ID=15,length=102531392>
##contig=<ID=16,length=90354753>
##contig=<ID=17,length=81195210>
##contig=<ID=18,length=78077248>
##c

The format option, __`-f`__ can be used to select what gets printed from your query command. For example, the following will print the position, reference base and alternate base for sample HG00131, separated by tabs:

In [5]:
bcftools query -f'%POS\t%REF\t%ALT\n' -s HG00131 1kg.bcf | head

23946362	G	A
23946710	C	T
23947133	C	T
23947416	C	T
23947451	T	G
23947981	T	C
23948049	A	G
23948150	G	A
23948593	G	A
23948726	C	T
[query_vcf] Error: cannot write to standard output


Finally, let's look at the __`-i`__ option. With this option we can select only sites for which a particular expression is true. For instance, if we only want to look at sites that have more than 2 alternate alleles, we can use the following expression (piped to `head` to only show a subset of the output):

In [6]:
bcftools query -f'%CHROM\t%POS\t%REF\t%ALT\n' -i 'AC[0]>2' 1kg.bcf | head

20	23946710	C	T
20	23947416	C	T
20	23947451	T	G
20	23948049	A	G
20	23948593	G	A
20	23948726	C	T
20	23949020	A	G
20	23949210	A	G
20	23949344	A	C
20	23949583	C	A
[query_vcf] Error: cannot write to standard output


We use __`-i`__ with the expression `AC[0]>2`. AC is an info field that holds the __a__llele __c__ount. Some fields can hold multiple values, so we use `AC[0]>2` to indicate that we are looking for the first value (this is zero indexed, and hence starts at 0 instead of 1), and that this value should be > 2. To format our output, we use __`-f`__ to specify that we want to print the chromosome name and position.

There is more information about expressions on the bcftools manual page [http://samtools.github.io/bcftools/bcftools.html#expressions](http://samtools.github.io/bcftools/bcftools.html#expressions)

### Exercises
Now, try and answer the following questions about the file 1kg.bcf in the data directory. For more information about the different usage options you can open the bcftools query manual page [http://samtools.github.io/bcftools/bcftools.html#query](http://samtools.github.io/bcftools/bcftools.html#query) in a web browser.

__Q19: What version of the human assembly do the coordinates refer to?__ 

In [19]:
bcftools view -h 1kg.bcf

## HG 37 

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##samtoolsVersion=1.2-29-gca44680+htslib-1.2.1-50-g66e5ef4
##samtoolsCommand=samtools mpileup -gu -t DP -I -f ref/human/GRCh37/hs37d5.fa -b bams.chr20.list
##reference=file:///ref/human/GRCh37/hs37d5.fa
##bcftools_callVersion=1.2-208-gcdcf7e1+htslib-1.2.1-158-g6876a07
##bcftools_callCommand=call -mv -Oz -o chr20.dp.vcf.gz
##contig=<ID=1,length=249250621>
##contig=<ID=2,length=243199373>
##contig=<ID=3,length=198022430>
##contig=<ID=4,length=191154276>
##contig=<ID=5,length=180915260>
##contig=<ID=6,length=171115067>
##contig=<ID=7,length=159138663>
##contig=<ID=8,length=146364022>
##contig=<ID=9,length=141213431>
##contig=<ID=10,length=135534747>
##contig=<ID=11,length=135006516>
##contig=<ID=12,length=133851895>
##contig=<ID=13,length=115169878>
##contig=<ID=14,length=107349540>
##contig=<ID=15,length=102531392>
##contig=<ID=16,length=90354753>
##contig=<ID=17,length=81195210>
##contig=<ID=18,length=78077248>
##c

__Q20: How many samples are there in the BCF?__

In [20]:
bcftools query -l 1kg.bcf | wc -l

50


__Q21: What is the genotype of the sample HG00107 at the position 20:24019472? (Hint: use the combination of -r, -s, and -f options)__

In [24]:
bcftools query -f'[%GT]\n' -H -r 20:24019472 -s HG00107 1kg.bcf


# [1]HG00107:GT
0/1


__Q22: How many positions are there with more than 10 alternate alleles? (Hint: use the -i filtering option)__

In [25]:
bcftools query -f'%CHROM\t%POS\t%REF\t%ALT\n' -i 'AC[0]>10' 1kg.bcf | head

20	23946710	C	T
20	23948049	A	G
20	23948593	G	A
20	23949020	A	G
20	23949210	A	G
20	23949344	A	C
20	23949996	A	G
20	23950681	T	G
20	23951354	G	A
20	23951741	G	A
[query_vcf] Error: cannot write to standard output


__Q23: In how many positions does HG00107 have a non-reference genotype and a read depth bigger than 10? (Hint: you can use pipes to combine bcftools queries)__   

In [28]:
bcftools view -s HG00107 -H -i 'DP>10' 1kg.bcf | bcftools view -s HG00107 -H -i 'AC[0]>10' 1kg.bcf | wc -l

4778
[main_vcfview] Error: cannot write to (null)


# QC assessment of NGS data
QC is an important part of any analysis. In this section we are going to look at some of the metrics and graphs that can be used to assess the QC of NGS data. 


## Base quality
[Illumina sequencing](https://en.wikipedia.org/wiki/Illumina_dye_sequencing) technology relies on sequencing by synthesis. One of the most common problems with this is __dephasing__. For each sequencing cycle, there is a possibility that the replication machinery slips and either incorporates more than one nucleotide or perhaps misses to incorporate one at all. The more cycles that are run (i.e. the longer the read length gets), the greater the accumulation of these types of errors gets. This leads to a heterogeneous population in the cluster, and a decreased signal purity, which in turn reduces the precision of the base calling. The figure below shows an example of this.

Because of dephasing, it is possible to have high-quality data at the beginning of the read but really low-quality data towards the end of the read. In those cases you can decide to trim off the low-quality reads, for example using a tool called [Trimmomatic](http://www.usadellab.org/cms/?page=trimmomatic).




## Quality control of fastq files

Download the required dataset.

In [29]:
wget https://github.com/cb2edu/CB2-101-BioComp/raw/2020/07-NGS_Intro_QC/data/rnaseq_data.tar.gz

--2021-11-14 13:39:27--  https://github.com/cb2edu/CB2-101-BioComp/raw/2020/07-NGS_Intro_QC/data/rnaseq_data.tar.gz
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/cb2edu/CB2-101-BioComp/2020/07-NGS_Intro_QC/data/rnaseq_data.tar.gz [following]
--2021-11-14 13:39:28--  https://raw.githubusercontent.com/cb2edu/CB2-101-BioComp/2020/07-NGS_Intro_QC/data/rnaseq_data.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25184845 (24M) [application/octet-stream]
Saving to: ‘rnaseq_data.tar.gz’

100%[======================================>] 25,184,845  82.8MB/s   in 0.3s   

2021-11-14 13:39:28 (82.8 M

Unzip the tarball

In [30]:
tar -xvzf rnaseq_data.tar.gz

rnaseq_data/
rnaseq_data/adrenal_R1.fq
rnaseq_data/adrenal_R2.fq
rnaseq_data/brain_R1.fq
rnaseq_data/brain_R2.fq
rnaseq_data/chr19.fa
rnaseq_data/human_chr19.gtf


FASTQC is software that is used to calculate the quality of the fastq data.

In [31]:
wget https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip
unzip fastqc_v0.11.9.zip

--2021-11-14 13:39:53--  https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip
Resolving www.bioinformatics.babraham.ac.uk (www.bioinformatics.babraham.ac.uk)... 149.155.133.4
Connecting to www.bioinformatics.babraham.ac.uk (www.bioinformatics.babraham.ac.uk)|149.155.133.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10249221 (9.8M) [application/zip]
Saving to: ‘fastqc_v0.11.9.zip’

100%[======================================>] 10,249,221  3.97MB/s   in 2.5s   

2021-11-14 13:39:56 (3.97 MB/s) - ‘fastqc_v0.11.9.zip’ saved [10249221/10249221]

Archive:  fastqc_v0.11.9.zip
  inflating: FastQC/cisd-jhdf5.jar   
   creating: FastQC/Configuration/
  inflating: FastQC/Configuration/adapter_list.txt  
  inflating: FastQC/Configuration/contaminant_list.txt  
  inflating: FastQC/Configuration/limits.txt  
  inflating: FastQC/fastqc           
  inflating: FastQC/fastqc_icon.ico  
   creating: FastQC/Help/
   creating: FastQC/Help/1 Introductio

   creating: FastQC/org/apache/
   creating: FastQC/org/apache/commons/
   creating: FastQC/org/apache/commons/math3/
   creating: FastQC/org/apache/commons/math3/analysis/
   creating: FastQC/org/apache/commons/math3/analysis/solvers/
  inflating: FastQC/org/apache/commons/math3/analysis/solvers/AbstractUnivariateSolver.class  
  inflating: FastQC/org/apache/commons/math3/analysis/solvers/AllowedSolution.class  
  inflating: FastQC/org/apache/commons/math3/analysis/solvers/BaseAbstractUnivariateSolver.class  
  inflating: FastQC/org/apache/commons/math3/analysis/solvers/BaseUnivariateSolver.class  
  inflating: FastQC/org/apache/commons/math3/analysis/solvers/BracketedUnivariateSolver.class  
  inflating: FastQC/org/apache/commons/math3/analysis/solvers/BrentSolver.class  
  inflating: FastQC/org/apache/commons/math3/analysis/solvers/UnivariateSolver.class  
  inflating: FastQC/org/apache/commons/math3/analysis/solvers/UnivariateSolverUtils.class  
  inflating: FastQC/org/apache/commo

  inflating: FastQC/uk/ac/babraham/FastQC/Dialogs/WelcomePanel.class  
  inflating: FastQC/uk/ac/babraham/FastQC/FastQCApplication$1.class  
  inflating: FastQC/uk/ac/babraham/FastQC/FastQCApplication.class  
  inflating: FastQC/uk/ac/babraham/FastQC/FastQCConfig.class  
  inflating: FastQC/uk/ac/babraham/FastQC/FastQCMenuBar.class  
   creating: FastQC/uk/ac/babraham/FastQC/FileFilters/
  inflating: FastQC/uk/ac/babraham/FastQC/FileFilters/BAMFileFilter.class  
  inflating: FastQC/uk/ac/babraham/FastQC/FileFilters/CasavaFastQFileFilter.class  
  inflating: FastQC/uk/ac/babraham/FastQC/FileFilters/FastQFileFilter.class  
  inflating: FastQC/uk/ac/babraham/FastQC/FileFilters/GobyFileFilter.class  
  inflating: FastQC/uk/ac/babraham/FastQC/FileFilters/MappedBAMFileFilter.class  
  inflating: FastQC/uk/ac/babraham/FastQC/FileFilters/SequenceFileFilter.class  
   creating: FastQC/uk/ac/babraham/FastQC/Graphs/
  inflating: FastQC/uk/ac/babraham/FastQC/Graphs/BaseGroup.class  
  inflating: F

Check the help of fastqc.

In [35]:
FastQC/fastqc -h



            FastQC - A high throughput sequence QC analysis tool

SYNOPSIS

	fastqc seqfile1 seqfile2 .. seqfileN

    fastqc [-o output dir] [--(no)extract] [-f fastq|bam|sam] 
           [-c contaminant file] seqfile1 .. seqfileN

DESCRIPTION

    FastQC reads a set of sequence files and produces from each one a quality
    control report consisting of a number of different modules, each one of 
    which will help to identify a different potential type of problem in your
    data.
    
    If no files to process are specified on the command line then the program
    will start as an interactive graphical application.  If files are provided
    on the command line then the program will run with no user interaction
    required.  In this mode it is suitable for inclusion into a standardised
    analysis pipeline.
    
    The options for the program as as follows:
    
    -h --help       Print this help file and exit
    
    -v --version    Print the version of the program and exit

In [36]:
mkdir -p rnaseq_quality
FastQC/fastqc -o rnaseq_quality -f fastq rnaseq_data/adrenal_R1.fq

Started analysis of adrenal_R1.fq
Approx 5% complete for adrenal_R1.fq
Approx 10% complete for adrenal_R1.fq
Approx 15% complete for adrenal_R1.fq
Approx 20% complete for adrenal_R1.fq
Approx 25% complete for adrenal_R1.fq
Approx 30% complete for adrenal_R1.fq
Approx 35% complete for adrenal_R1.fq
Approx 40% complete for adrenal_R1.fq
Approx 45% complete for adrenal_R1.fq
Approx 50% complete for adrenal_R1.fq
Approx 55% complete for adrenal_R1.fq
Approx 60% complete for adrenal_R1.fq
Approx 65% complete for adrenal_R1.fq
Approx 70% complete for adrenal_R1.fq
Approx 75% complete for adrenal_R1.fq
Approx 80% complete for adrenal_R1.fq
Approx 85% complete for adrenal_R1.fq
Approx 90% complete for adrenal_R1.fq
Approx 95% complete for adrenal_R1.fq
Analysis complete for adrenal_R1.fq


  Open the analysis file.

In [ ]:
firefox rnaseq_quality/adrenal_R1_fastqc.html

A good example:
http://www.bioinformatics.babraham.ac.uk/projects/fastqc/bad_sequence_fastqc.html

A bad example:
http://www.bioinformatics.babraham.ac.uk/projects/fastqc/bad_sequence_fastqc.html

## Read trimming
We will use Trimmomatic for read trimming and adapter removal. Download Trimmomatic from: http://www.usadellab.org/cms/?page=trimmomatic
Generally, Illumina adapters are of two types: Nextera for WGS and exome sequencing and Truseq from RNAseq. We need to keep that in mind and provide the for trimming.

We used TruSeq3-PE adapters for clipping. Some other options are:
1. Adapters will have max 2 mismatches and will be clipped if a score of 30 is reached. 2. Remove leading and trailing N bases if quality is below 2.
3. Move with a 4 bp window and cutting where the average quality falls below 15.
4. After trmming remove all sequences whose length is below 30.

In [37]:
wget http://www.usadellab.org/cms/uploads/supplementary/Trimmomatic/Trimmomatic-0.39.zip
unzip Trimmomatic-0.39.zip

--2021-11-14 14:33:24--  http://www.usadellab.org/cms/uploads/supplementary/Trimmomatic/Trimmomatic-0.39.zip
Resolving www.usadellab.org (www.usadellab.org)... 199.195.140.36
Connecting to www.usadellab.org (www.usadellab.org)|199.195.140.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 133596 (130K) [application/zip]
Saving to: ‘Trimmomatic-0.39.zip’

100%[======================================>] 133,596      356KB/s   in 0.4s   

2021-11-14 14:33:25 (356 KB/s) - ‘Trimmomatic-0.39.zip’ saved [133596/133596]

Archive:  Trimmomatic-0.39.zip
   creating: Trimmomatic-0.39/
  inflating: Trimmomatic-0.39/LICENSE  
  inflating: Trimmomatic-0.39/trimmomatic-0.39.jar  
   creating: Trimmomatic-0.39/adapters/
  inflating: Trimmomatic-0.39/adapters/NexteraPE-PE.fa  
  inflating: Trimmomatic-0.39/adapters/TruSeq2-PE.fa  
  inflating: Trimmomatic-0.39/adapters/TruSeq2-SE.fa  
  inflating: Trimmomatic-0.39/adapters/TruSeq3-PE-2.fa  
  inflating: Trimmomatic-0.39/adapters/

In [38]:
java -jar Trimmomatic-0.39/trimmomatic-0.39.jar PE \
rnaseq_data/adrenal_R1.fq rnaseq_data/adrenal_R2.fq \
adrenal_R1_trim.fq adrenal_R1_unmapped.fq \
adrenal_R2_trim.fq adrenal_R2_unmapped.fq \
ILLUMINACLIP:Trimmomatic-0.39/adapters/TruSeq3-PE.fa:2:30:10 \
LEADING:3 TRAILING:3 MINLEN:30

TrimmomaticPE: Started with arguments:
 rnaseq_data/adrenal_R1.fq rnaseq_data/adrenal_R2.fq adrenal_R1_trim.fq adrenal_R1_unmapped.fq adrenal_R2_trim.fq adrenal_R2_unmapped.fq ILLUMINACLIP:Trimmomatic-0.39/adapters/TruSeq3-PE.fa:2:30:10 LEADING:3 TRAILING:3 MINLEN:30
Using PrefixPair: 'TACACTCTTTCCCTACACGACGCTCTTCCGATCT' and 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT'
ILLUMINACLIP: Using 1 prefix pairs, 0 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Read Pairs: 50121 Both Surviving: 47073 (93.92%) Forward Only Surviving: 2271 (4.53%) Reverse Only Surviving: 571 (1.14%) Dropped: 206 (0.41%)
TrimmomaticPE: Completed successfully


## Generating QC stats of bam files
Now let's try this out! We will generate QC stats for two lanes of Illumina paired-end sequencing data from yeast. The reads have already been aligned to the [Saccromyces cerevisiae reference genome](ftp://ftp.ensembl.org/pub/current_fasta/saccharomyces_cerevisiae/dna) to produce the BAM file lane1.sorted.bam.

Now we will use __`samtools stats`__ to generate the stats for the primary alignments. The option __`-f`__ can be used to filter reads with specific tags, while __`-F`__ can be used to _filter out_ reads with specific tags. The following command will include only primary alignments:

In [3]:
samtools stats -F SECONDARY NA20538_sorted.bam \
    > NA20538.sorted.bam.bchk

Have a look at the first 47 lines of the statistics file that was generated:

In [4]:
head -n 47 NA20538.sorted.bam.bchk

# This file was produced by samtools stats (1.12+htslib-1.12) and can be plotted using plot-bamstats
# This file contains statistics for all reads.
# The command line was:  stats -F SECONDARY NA20538_sorted.bam
# CHK, Checksum	[2]Read Names	[3]Sequences	[4]Qualities
# CHK, CRC32 of reads which passed filtering followed by addition (32bit overflow)
CHK	14ad5a44	b9736b75	d4f02655
# Summary Numbers. Use `grep ^SN | cut -f 2-` to extract this part.
SN	raw total sequences:	347367	# excluding supplementary and secondary reads
SN	filtered sequences:	0
SN	sequences:	347367
SN	is sorted:	1
SN	1st fragments:	174411
SN	last fragments:	172956
SN	reads mapped:	323966
SN	reads mapped and paired:	298906	# paired-end technology bit set + both mates mapped
SN	reads unmapped:	23401
SN	reads properly paired:	286546	# proper-pair bit set
SN	reads paired:	345708	# paired-end technology bit set
SN	reads duplicated:	3212	# PCR or optical duplicate bit set
SN	reads MQ0:	17779	# mapped and MQ=0
SN	reads QC fai

This file contains a number of useful stats that we can use to get a better picture of our data, and it can even be plotted with __`plot-bamstats`__, as you will see soon. First let's have a closer look at some of the different stats. Each part of the file starts with a `#` followed by a description of the section and how to extract it from the file. Let's have a look at all the sections in the file:

In [5]:
grep ^'#' NA20538.sorted.bam.bchk | grep 'Use'

# Summary Numbers. Use `grep ^SN | cut -f 2-` to extract this part.
# First Fragment Qualities. Use `grep ^FFQ | cut -f 2-` to extract this part.
# Last Fragment Qualities. Use `grep ^LFQ | cut -f 2-` to extract this part.
# GC Content of first fragments. Use `grep ^GCF | cut -f 2-` to extract this part.
# GC Content of last fragments. Use `grep ^GCL | cut -f 2-` to extract this part.
# ACGT content per cycle. Use `grep ^GCC | cut -f 2-` to extract this part. The columns are: cycle; A,C,G,T base counts as a percentage of all A/C/G/T bases [%]; and N and O counts as a percentage of all A/C/G/T bases [%]
# ACGT content per cycle, read oriented. Use `grep ^GCT | cut -f 2-` to extract this part. The columns are: cycle; A,C,G,T base counts as a percentage of all A/C/G/T bases [%]
# ACGT content per cycle for first fragments. Use `grep ^FBC | cut -f 2-` to extract this part. The columns are: cycle; A,C,G,T base counts as a percentage of all A/C/G/T bases [%]; and N and O counts as a percenta

### Summary Numbers (SN)
This initial section contains a summary of the alignment and includes some general statistics. In particular, you can see how many bases mapped, and how much of the genome that was covered.


In [6]:
grep ^'SN' NA20538.sorted.bam.bchk 

SN	raw total sequences:	347367	# excluding supplementary and secondary reads
SN	filtered sequences:	0
SN	sequences:	347367
SN	is sorted:	1
SN	1st fragments:	174411
SN	last fragments:	172956
SN	reads mapped:	323966
SN	reads mapped and paired:	298906	# paired-end technology bit set + both mates mapped
SN	reads unmapped:	23401
SN	reads properly paired:	286546	# proper-pair bit set
SN	reads paired:	345708	# paired-end technology bit set
SN	reads duplicated:	3212	# PCR or optical duplicate bit set
SN	reads MQ0:	17779	# mapped and MQ=0
SN	reads QC failed:	0
SN	non-primary alignments:	0
SN	supplementary alignments:	0
SN	total length:	17953503	# ignores clipping
SN	total first fragment length:	9018082	# ignores clipping
SN	total last fragment length:	8935421	# ignores clipping
SN	bases mapped:	17019151	# ignores clipping
SN	bases mapped (cigar):	16337323	# more accurate
SN	bases trimmed:	0
SN	bases duplicated:	224066
SN	mismatches:	158471	# from NM fields
SN	error rate:	9.699937e-03	# mismatch

### Exercises

Now look at the output and try to answer the questions below.

__Q2: What is the total number of reads?__

Total reads mapped is 323966

__Q3: What proportion of the reads were mapped?__

proportion of raw total sequences to reads mapped is 347367 / 323966 or 1.07

__Q4: How many pairs were mapped to a different chromosome?__

4055 were mapped to different chromosomes

__Q5: What is the insert size mean and standard deviation?__

insertion size mean is 190.3 with standard deviation of 136.4

__Q6: How many reads were paired properly?__

reads properly paired:	286546

__Q7: How many reads have zero mapping quality?__

17779 have a mapping quality of zero

__Q8: Which reads (forward or reverse) have higher base quality on average?__  

### Generating QC plots
Finally, we will create some QC plots from the output of the stats command using the command __plot-bamstats__ which is included in the samtools package: 

In [9]:
plot-bamstats -p NA20538-plots/ NA20538.sorted.bam.bchk

Now in your web browser open the file NA20538-plots/index.html to view the QC information. 

__Congratulations__ you have reached the end of the Data formats and QC tutorial! 